In [1]:
import pandas as pd

In [2]:
df_fr = pd.read_csv("../data/french_data.csv").dropna()
df_be = pd.read_csv("../data/belgium_data.csv").dropna()
df_eu = pd.read_csv("../data/european_data.csv").dropna()

In [3]:
df_eu

,Exposure,Power,DriverAge,Fuel_type,Density,Sex,Sinistre
0,0.487671,14,77,0,250.0,1,0
1,0.019178,12,40,0,429.0,1,0
2,0.032877,14,75,0,86.0,1,0
3,0.043836,13,48,0,429.0,1,0
4,0.046575,12,54,0,429.0,0,0
...,...,...,...,...,...,...,...
2373192,0.246575,14,40,0,784.0,0,0
2373193,0.756164,14,52,0,296.0,1,0
2373194,0.008219,1,46,0,296.0,0,0
2373195,0.723288,15,25,0,296.0,1,0


In [12]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler

# Chargement des données
df_fr = pd.read_csv("../data/french_data.csv").dropna()
df_be = pd.read_csv("../data/belgium_data.csv").dropna()
df_eu = pd.read_csv("../data/european_data.csv").dropna()

# Création des variables binaires sans relation d'ordre
dataframes = [df_fr, df_be, df_eu]
for df in dataframes:
    # Variables pour le sexe
    df['Homme'] = df['Sex'].astype(int)
    df['Femme'] = 1 - df['Sex'].astype(int)
    
    # Variables pour le type de carburant
    df['Diesel'] = df['Fuel_type'].astype(int)
    df['Regular'] = 1 - df['Fuel_type'].astype(int)

In [7]:
df_fr

,Exposure,Power,DriverAge,Fuel_type,Density,Sex,Sinistre,Homme,Femme,Diesel,Regular
0,0.09000,43,46,1,76,0,0,0,1,1,0
1,0.84000,43,46,1,76,1,0,1,0,1,0
2,0.52000,40,38,0,3003,1,0,1,0,0,1
3,0.45000,40,38,0,3003,1,0,1,0,0,1
4,0.15000,43,41,1,60,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...
1091177,0.00274,4,54,0,3317,0,0,0,1,0,1
1091178,0.00274,4,41,0,9850,1,0,1,0,0,1
1091179,0.00274,6,45,1,1323,1,0,1,0,1,0
1091180,0.00274,4,60,0,95,1,0,1,0,0,1


In [13]:
# Normalisation des variables numériques
cols_to_scale = ['Power', 'DriverAge', 'Density']
for df in dataframes:
    scaler = MinMaxScaler()
    df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])

In [14]:
df_fr

,Exposure,Power,DriverAge,Fuel_type,Density,Sex,Sinistre,Homme,Femme,Diesel,Regular
0,0.09000,0.336207,0.341463,1,0.002778,0,0,0,1,1,0
1,0.84000,0.336207,0.341463,1,0.002778,1,0,1,0,1,0
2,0.52000,0.310345,0.243902,0,0.111189,1,0,1,0,0,1
3,0.45000,0.310345,0.243902,0,0.111189,1,0,1,0,0,1
4,0.15000,0.336207,0.280488,1,0.002185,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...
1091177,0.00274,0.000000,0.439024,0,0.122819,0,0,0,1,0,1
1091178,0.00274,0.000000,0.280488,0,0.364791,1,0,1,0,0,1
1091179,0.00274,0.017241,0.329268,1,0.048965,1,0,1,0,1,0
1091180,0.00274,0.000000,0.512195,0,0.003482,1,0,1,0,0,1


In [16]:
# Régression logistique et affichage des résultats avec summary
for df_name, df in zip(['df_fr', 'df_be', 'df_eu'], dataframes):
    # Sélection des variables (éviter la multicolinéarité)
    X = df[['Power', 'DriverAge', 'Density', 'Homme', 'Diesel']]
    X = sm.add_constant(X)
    y = df['Sinistre']
    
    # Ajustement du modèle
    model = sm.Logit(y, X).fit()
    
    # Affichage des résultats avec summary
    print(f"\nRésultats pour {df_name}:")
    print(model.summary())


Optimization terminated successfully.
         Current function value: 0.184140
         Iterations 7

Résultats pour df_fr:
                           Logit Regression Results                           
Dep. Variable:               Sinistre   No. Observations:              1091182
Model:                          Logit   Df Residuals:                  1091176
Method:                           MLE   Df Model:                            5
Date:                Tue, 06 May 2025   Pseudo R-squ.:                0.001900
Time:                        14:25:02   Log-Likelihood:            -2.0093e+05
converged:                       True   LL-Null:                   -2.0131e+05
Covariance Type:            nonrobust   LLR p-value:                4.105e-163
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.0211      0.013   -228.056      0.000      -3.047      -2.995
Power 

In [17]:
df_eu.to_csv('../data/european_data.csv', index=False)
df_fr.to_csv('../data/french_data.csv', index=False)
df_be.to_csv('../data/belgium_data.csv', index=False)